In [1]:
import dimod
from dimod import BinaryQuadraticModel
import dwave
from dwave.samplers import SimulatedAnnealingSampler
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite

import networkx as nx
import minorminer

import numpy as np
from matplotlib import pyplot as plt
from tqdm import notebook

In [2]:
def annealing(H,num_shot,sampler=None,embedding=None,anneal_time=None,chain_strength=10):
    if type(sampler) == None or type(embedding) == None:
        sampler_  = SimulatedAnnealingSampler()
        sampleset = sampler_.sample(H,num_reads=num_shot,answer_mode='raw')
    else:
        sampler_     = FixedEmbeddingComposite(sampler,embedding)
#         schedule  = [[0.,0.],[anneal_time*(1/3),0.5],[anneal_time*(2/3),0.5],[anneal_time,1.0]] 
        sampleset    = sampler_.sample(H,num_reads=num_shot,chain_strength=chain_strength,\
                                    answer_mode='raw',annealing_time=anneal_time)
#         sampleset    = sampler_.sample(H,num_reads=num_shot,chain_strength=chain_strength,\
#                                     answer_mode='raw',anneal_schedule=schedule)
    return sampleset

def check_chain_break(confi,embedding):
    for chains in embedding.values():
        lst = []
        for chain in chains:
            lst.append(confi[chain])
        if np.abs(np.mean(lst)) != 1:
            return True
    return False

def defaultJc(H,prefactor=1.414):
    num_interactions = H.num_interactions
    quadratic_array = np.fromiter(H.quadratic.values(), dtype=float, count=num_interactions)    
    squared_j = quadratic_array**2
    rms = np.sqrt(squared_j.sum() / num_interactions)
    avg_degree = H.degrees(array=True).mean()
    Jc = prefactor * rms * np.sqrt(avg_degree)   
    return Jc

def det_chain_strength(H,sampler,embedding,cnst=1.6,num_shot=100,anneal_time=1000,prefactor=1.414,num=10):
    # embd BQM을 위한 embedding
    embedding_ = {}
    for qubit, chains in embedding.items():
        for chain in chains:
            embedding_[chain] = (chain,)      
    # default chain strength
    init_Jc = defaultJc(H,prefactor)
    Jc_lst  = np.linspace(init_Jc/10,init_Jc,num=num)
    
    for j in np.arange(Jc_lst.shape[0]):
        Jc        = Jc_lst[j]
        H_        = dwave.embedding.embed_bqm(H,embedding,pegasus_graph,chain_strength=Jc) 
        sampleset = annealing(H_,num_shot,sampler,embedding,anneal_time)
        confi     = sampleset.first.sample
        if check_chain_break(confi,embedding) == False:
            Jc_ = Jc_lst[j-1]
            break    
    return cnst * Jc_          

In [4]:
sampler            = DWaveSampler(solver='Advantage_system4.1',\
                                  token='token')
print("Connected to sampler", sampler.solver.name)
pegasus_properties = sampler.properties
pegasus_graph      = sampler.to_networkx_graph()

Connected to sampler Advantage_system4.1


In [4]:
L        = 6
J1       = 10
J2       = 4
num_shot = 1000
H        = hamiltonianJ1J2(L,J1,J2)
G        = dimod.to_networkx_graph(H)

In [5]:
embd_num      = 1000
embd_lst      = []
chain_num_lst = []

for _ in notebook.tqdm(np.arange(embd_num)):
    embd      = find_embedding(G,pegasus_graph)
    chain_num = ret_num_chain(embd)
    embd_lst.append(embd)
    chain_num_lst.append(chain_num)

  0%|          | 0/1000 [00:00<?, ?it/s]